In [20]:
DATASET_DIR='./Users\Dimble Scaria\Desktop\Automated-Essay--Scoring\data'
import os
import pandas as pd
x=pd.read_csv(os.path.join('training_set_rel3.tsv'),sep='\t',encoding='ISO-8859-1')
y=x['domain1_score']
y=x.dropna(axis=1)
x=x.drop(columns=['rater1_domain1', 'rater2_domain1'])
x.head()

,essay_id,essay_set,essay,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",NaN,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",NaN,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",NaN,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",NaN,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",NaN,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec


def essaytowordlist(essay_v,remove_stopwords):
    essay_v=re.sub("[^a-zA-Z]"," ",essay_v)
    words=essay_v.lower().split()
    if remove_stopwords:
        stops=set(stopwords.words("english"))
        words=[w for w in words if not w in stops]
        return words
        
def essaytosentences(essay_v,remove_stopwords):
    tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentence=tokenizer.tokenize(essay_v.strip())
    sentences=[]
    for r in raw_sentence:
        if len(r)>0:
            sentences.append(essaytowordlist(r,remove_stopwords))
            return sentences
        
import numpy as np
def makefeaturevec(words,model,num_features):
    featurevec=np.zeros((num_features),dtype="float32")
    num_words=0
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words+=1
            featurevec=np.add(featurevec,model[word])
        featurevec=np.divide(featurevec,num_words)
        #print(featurevec)
    return featurevec 

def getavgfeaturevec(essays,model,num_features):
    counter=0
    essayfeaturevecs=np.zeros((len(essays),num_features),dtype='float32')
    for essay in essays:
        essayfeaturevecs[counter]=makefeaturevec(essay,model,num_features)
        counter+=1
    return essayfeaturevecs


# model

In [22]:

from tensorflow.keras.layers import Embedding,Lambda,Flatten,LSTM,Dense,Dropout
from tensorflow.keras.models import Sequential,load_model,model_from_config
import tensorflow.keras.backend as k

def get_model():
    model=Sequential()
    model.add(LSTM(300,dropout=0.4,recurrent_dropout=0.4,input_shape=[1,300],return_sequences=True))
    model.add(LSTM(64,recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='relu'))
    
    model.compile(loss='mean_squared_error',optimizer='rmsprop',metrics=['mae'])
    model.summary
    return model



In [23]:
from sklearn.cross_validation import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

cv=KFold(len(x),n_folds=5,shuffle=True)
results=[]
y_pred_list=[]

count=1

In [19]:
testcv

array([    0,     9,    10, ..., 12964, 12966, 12974])

In [15]:

for traincv,testcv in cv:
    X_test,X_train,y_test,y_train=x.iloc[testcv],x.iloc[traincv],y.iloc[testcv],y.iloc[traincv]
    
    train_essays=X_train['essay']
    test_essays=X_test['essay']
    
    sentences=[]
    
    for essay in train_essays:
        sentences+=essaytosentences(essay,remove_stopwords=True)
        
    num_features=300
    min_word_count=40
    num_workers=4
    context=10
    downsampling=1e-3
    
    print("Training Word2Vec Model...")
    model=Word2Vec(sentences,workers=num_workers,size=num_features, min_count = min_word_count, window = context, sample = downsampling)
   
    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
   
    clean_train_essays = []
    for essay_v in train_essays:
        clean_train_essays.append(essaytowordlist(essay_v,remove_stopwords=True))    
    traindatavecs=getavgfeaturevec(clean_train_essays,model,num_features)

    
    clean_test_essays=[]
    for essay_v in train_essays:
        clean_test_essays.append(essaytowordlist(essay_v,remove_stopwords=True))
    testdatavecs=getavgfeaturevec(clean_train_essays,model,num_features)
    
    
    traindatavecs=np.array(traindatavecs)
    testdatavecs=np.array(testdatavecs)
    traindatavecs=np.reshape(traindatavecs,(traindatavecs.shape[0],1,traindatavecs.shape[1]))
    testdatavecs=np.reshape(testdatavecs,(testdatavecs.shape[0],1,testdatavecs.shape[1]))
    
   
    #Dense(units = 3, activation = "relu")

    #y_train = np.asarray(y_train,order=None)
    
    #pd.get_dummeis(pd.Series(y_train)).to_numpy()
   
    
    lstm_model=get_model()
    lstm_model.fit(traindatavecs,y_train.values,batch_size=64,epochs=50)
    y_pred=(lstm_model.predict(testdatavecs))
    print("hello")
    if count==5:
        lstm_model.save('./model_weights/final_lstm.h5')
    y_pred = np.around(y_pred)
    
    
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1

    
    
    



Training Word2Vec Model...


C:\Users\Dimble Scaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\Dimble Scaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide


ValueError: A target array with shape (10380, 6) was passed for an output of shape (None, 1) while using as loss `mean_squared_error`. This loss expects targets to have the same shape as the output.

In [13]:
pd.DataFrame(np.random.random([2,4])).values

array([[0.64588701, 0.03755483, 0.86325473, 0.32198633],
       [0.56631841, 0.36699907, 0.82809852, 0.88892941]])

In [ ]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

In [21]:
import h5py as hp


In [23]:
f=hp.File('final_lstm.h5','r')

In [25]:
list(f.keys())

['model_weights', 'optimizer_weights']

In [26]:
dset=f['model_weights']

In [31]:
type(dset)

h5py._hl.group.Group